In [1]:
from midiutil import MIDIFile
import pandas as pd
from music21 import midi
import numpy as np
import markov_bach

midi_note_converter = {'pitch_1':60,
             'pitch_2':61,
             'pitch_3':62,
             'pitch_4':63,
             'pitch_5':64,
             'pitch_6':65,
             'pitch_7':66,
             'pitch_8':67,
             'pitch_9':68,
             'pitch_10':69,
             'pitch_11':70,
             'pitch_12':71}


def string2int(chord):
    return [int(i) for i in chord.strip('[').strip(']').split(', ')]

def chord2midi(chord_bass_tuple):
    """
    Convert (chord name, bass note) tuple to a (chord as midi list, bass as midi note) tuple
    """
    chord, bass = chord_bass_tuple
    return chords[chord][bass], bass_converter[bass]


df = pd.read_csv('assets/encoded.csv')
note_columns = df.columns[2:14]

# the 'chords' dict maps a chord (F_M) to a dictionary, where the keys are bass notes. Each bass note maps
# to a single list of MIDI pitches that represent the notes in the chord
chords = {}
for i, row in df.iterrows():
    chord = row['chord_label']
    bass = row['bass']
    if chord not in chords:
        chords[chord] = {}
    if bass not in chords[chord].keys():
        notes = []
        for col in note_columns:
            if row[col] == 1:
                note = midi_note_converter[col]
                notes.append(note)
        chords[chord][bass] = str(notes)


bass_converter = {'C': 48,
                  'C#': 49,
                  'Db': 49,
                  'D': 50,
                  'Eb': 51,
                  'D#': 51,
                  'E': 52,
                  'F': 53,
                  'F#': 54,
                  'G': 55,
                  'G#': 56,
                  'Ab': 56,
                  'A': 57,
                  'Bb': 58,
                  'A#': 58,
                  'B': 59
}

start_chord = 'D_M'
start_bass = 'D'

chord_model = markov_bach.Chain(NONWORD=(start_chord, start_bass), NPREF=4)
meter_model = markov_bach.Chain(NONWORD=3, NPREF=2)
#bass = Chain(NONWORD='F', NPREF=4)

# create markov chains from all chorales
for chorale in df['choral_ID'].unique():
    d = df[df['choral_ID'] == chorale]  # select only the rows from a single chorale
    chord_stream = [(i['chord_label'], i['bass']) for k, i in d.iterrows()]  # sequential list of all chord/bass tuples
    chord_model.build(chord_stream)
    meter_stream = np.array(d['meter'], dtype='str')
    meter_model.build(meter_stream)

# generate chords and durations
generated_chords = chord_model.generate(first=(start_chord, start_bass))
generated_meter = meter_model.generate(first=3)

# convert chord names to midi note lists
# generated_chords[i] is a tuple: (chord as midi list, midi bass note)
generated_chords_midi = [chord2midi(x) for x in generated_chords]

# create MIDI
track = 0
channel = 0
time = 0
duration = 1
tempo = 60
volume = 100

MIDI = MIDIFile(2)

t = 0

start_c = string2int(chords[start_chord][start_bass])
start_b = bass_converter[start_bass]
duration = 4
for c in start_c:
    MIDI.addNote(track, channel, c, t, duration, volume)
MIDI.addNote(track, channel, start_b, t, duration, volume)
t += duration

for i in range(len(generated_chords_midi)):
    chord, bass = generated_chords_midi[i]
    chord = string2int(chord)
    duration = int(generated_meter[i])

    for note in chord:
        MIDI.addNote(track, channel, note, t, duration, volume)

    MIDI.addNote(track, channel, bass, t, duration, volume - 10)

    t += duration


with open("generated_all_midi.mid", 'wb') as output_file:
    MIDI.writeFile(output_file)

mf = midi.MidiFile()
mf.open('generated_all_midi.mid')
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)

# print((start_c, start_b), generated_chords)
# score = converter.parse('generated_all_midi.mid')
# score.show()
s.show('midi')
